In [1]:
import pandas as pd
import os
import anndata as ad
import numpy as np
import scanpy as sc

In [2]:
# TMP
PROJECT_PATH = '/home/fichtner/projects/footprintQTL'
DATA_PATH = '/omics/groups/OE0540/internal/projects/HCA_organoid_2/cemm_sabrina-20Jul2022/'
RNA_AD = 'outputs_allsamples/sabrina_allsamples_rna_final_after_atac.h5ad'

import sys
sys.path.append(PROJECT_PATH + "/code")
from helpers.helpers import ct_format, get_anndata_coldata

cells_coldata = get_anndata_coldata(os.path.join(DATA_PATH, RNA_AD))[0]
dids_borgs = set(cells_coldata['donor_id'].unique().tolist())
del cells_coldata

In [3]:
# Exclude for some weird reason, samples in QC but not in other files
excl = {'SAMEA2474458', 'SAMEA2555012'}

# Chromatin accessibility (phenotype)

In [4]:
borgs_tile_mat = ad.read_h5ad('../data/datasets/hca_brain-organoids_processed/chromatin_accessibility/peak-matrix_rna-qc-cells_norm-reads-in-tss.h5ad')
borgs_tile_mat

/home/fichtner/.conda/envs/ian/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 288900 × 736845
    obs: 'BlacklistRatio', 'nDiFrags', 'nFrags', 'nMonoFrags', 'nMultiFrags', 'NucleosomeRatio', 'PassQC', 'PromoterRatio', 'ReadsInBlacklist', 'ReadsInPromoter', 'ReadsInTSS', 'Sample', 'TSSEnrichment', 'celltype', 'cellType', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'donor_id', 'clone', 'organoid', 'donor', 'leiden', 'stressed_vs_unstressed_celltypes', 'Batch', 'ReadsInPeaks', 'FRIP', 'barcode'
    var: 'chr', 'start', 'end', 'strand', 'peak_name', 'score', 'GC', 'nearest_gene', 'peak_type'

In [5]:
dids_tiles = set(borgs_tile_mat.obs['donor_id'])

In [6]:
print(f'len dids_tiles: {len(dids_tiles)}')
print(f'len dids_borgs: {len(dids_borgs)}')
print(dids_tiles - dids_borgs)
print(dids_borgs - dids_tiles)

len dids_tiles: 72
len dids_borgs: 72
set()
set()


## Pilot: Midbrain EN & DRD4

### Peak locations

In [7]:
ph_pilot = borgs_tile_mat[borgs_tile_mat.obs['celltype'] == 'Midbrain EN', borgs_tile_mat.var['nearest_gene'] == 'DRD4']
ph_pilot

View of AnnData object with n_obs × n_vars = 24950 × 23
    obs: 'BlacklistRatio', 'nDiFrags', 'nFrags', 'nMonoFrags', 'nMultiFrags', 'NucleosomeRatio', 'PassQC', 'PromoterRatio', 'ReadsInBlacklist', 'ReadsInPromoter', 'ReadsInTSS', 'Sample', 'TSSEnrichment', 'celltype', 'cellType', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'donor_id', 'clone', 'organoid', 'donor', 'leiden', 'stressed_vs_unstressed_celltypes', 'Batch', 'ReadsInPeaks', 'FRIP', 'barcode'
    var: 'chr', 'start', 'end', 'strand', 'peak_name', 'score', 'GC', 'nearest_gene', 'peak_type'

In [8]:
ph_pilot.obs

,BlacklistRatio,nDiFrags,nFrags,nMonoFrags,nMultiFrags,NucleosomeRatio,PassQC,PromoterRatio,ReadsInBlacklist,ReadsInPromoter,...,donor_id,clone,organoid,donor,leiden,stressed_vs_unstressed_celltypes,Batch,ReadsInPeaks,FRIP,barcode
index_name,,,,,,,,,,,,,,,,,,,,,
GAAAGCCAGGCGAATA-1_sSL0107,0.004526,10046,29714,15899,3769,0.868923,1,0.221730,269,13177,...,SAMEA2547622,HPSI1013i-kuxp_1,kuxp_sSL0107,kuxp,27,unstressed,sabrina_sSL0107,38523,0.648732,GAAAGCCAGGCGAATA-1
TGTAAAGCAGGAATCG-1_sSL0107,0.004622,12002,28885,12232,4651,1.361429,1,0.215683,267,12460,...,SAMEA3854308,HPSI0414i-oaqd_2,oaqd_sSL0107,oaqd,10,unstressed,sabrina_sSL0107,34782,0.602494,TGTAAAGCAGGAATCG-1
CGTACGGGTAAAGCGG-1_sSL0107,0.005919,9443,28552,15761,3348,0.811560,1,0.130008,338,7424,...,SAMEA3854308,HPSI0414i-oaqd_2,oaqd_sSL0107,oaqd,23,unstressed,sabrina_sSL0107,21616,0.378882,CGTACGGGTAAAGCGG-1
AGTGTGGCAGTCTATG-1_sSL0107,0.004869,8440,25157,13579,3138,0.852640,1,0.129984,245,6540,...,SAMEA2547622,HPSI1013i-kuxp_1,kuxp_sSL0107,kuxp,27,unstressed,sabrina_sSL0107,21574,0.429984,AGTGTGGCAGTCTATG-1
TTTGACTTCGATTTGA-1_sSL0107,0.003937,10746,24767,9708,4313,1.551195,1,0.153854,195,7621,...,SAMEA2547622,HPSI1013i-kuxp_1,kuxp_sSL0107,kuxp,27,unstressed,sabrina_sSL0107,26168,0.528860,TTTGACTTCGATTTGA-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GGAACCACATGGAGGC-1_sSL0174,0.009461,355,1057,556,146,0.901079,1,0.270104,20,571,...,SAMEA2474458,HPSI1013i-pamv_3,pamv_sSL0174,pamv,10,unstressed,sabrina_sSL0174,1364,0.645222,GGAACCACATGGAGGC-1
TACAGCTAGCTAATTG-1_sSL0174,0.007641,328,1047,609,110,0.719212,1,0.085005,16,178,...,SAMEA2518334,HPSI1013i-wuye_3,wuye_sSL0174,wuye,10,unstressed,sabrina_sSL0174,599,0.286055,TACAGCTAGCTAATTG-1
TTGTCCCAGGAGTAAT-1_sSL0174,0.008671,327,1038,607,104,0.710049,1,0.237476,18,493,...,SAMEA2518334,HPSI1013i-wuye_3,wuye_sSL0174,wuye,9,unstressed,sabrina_sSL0174,1051,0.507239,TTGTCCCAGGAGTAAT-1


In [9]:
ph_pilot.var

,chr,start,end,strand,peak_name,score,GC,nearest_gene,peak_type
447931,chr11,632981,633481,*,chr11:632981:633481:501:*:276,65.95070,0.4750,DRD4,Distal
447932,chr11,635503,636003,*,chr11:635503:636003:501:*:277,3.84324,0.5309,DRD4,Promoter
447933,chr11,636277,636777,*,chr11:636277:636777:501:*:278,2.55075,0.6607,DRD4,Promoter
447934,chr11,636987,637487,*,chr11:636987:637487:501:*:279,16.21060,0.7605,DRD4,Promoter
447935,chr11,637699,638199,*,chr11:637699:638199:501:*:280,15.83320,0.6567,DRD4,Intronic
447936,chr11,638698,639198,*,chr11:638698:639198:501:*:281,132.30600,0.5629,DRD4,Intronic
447937,chr11,639530,640030,*,chr11:639530:640030:501:*:282,6.99449,0.7804,DRD4,Exonic
447938,chr11,640244,640744,*,chr11:640244:640744:501:*:283,8.31427,0.6806,DRD4,Exonic
447939,chr11,641028,641528,*,chr11:641028:641528:501:*:284,9.42847,0.6786,DRD4,Distal
447940,chr11,641831,642331,*,chr11:641831:642331:501:*:285,9.94587,0.5988,DRD4,Distal


In [10]:
ph_pilot.var[['peak_name', 'chr', 'start', 'end']].to_csv("../data/datasets/hca_brain-organoids_processed/covariates/peak_locations.tsv", sep='\t', index=False)

### Peak CA values

In [11]:
ph_pilot_df = ph_pilot.to_df().copy()
ph_pilot_df = ph_pilot_df.rename(columns=ph_pilot.var['peak_name'].to_dict())
ph_pilot_df

,chr11:632981:633481:501:*:276,chr11:635503:636003:501:*:277,chr11:636277:636777:501:*:278,chr11:636987:637487:501:*:279,chr11:637699:638199:501:*:280,chr11:638698:639198:501:*:281,chr11:639530:640030:501:*:282,chr11:640244:640744:501:*:283,chr11:641028:641528:501:*:284,chr11:641831:642331:501:*:285,...,chr11:645521:646021:501:*:289,chr11:650440:650940:501:*:290,chr11:655195:655695:501:*:291,chr11:657366:657866:501:*:292,chr11:658442:658942:501:*:293,chr11:659910:660410:501:*:294,chr11:661013:661513:501:*:295,chr11:662642:663142:501:*:296,chr11:663338:663838:501:*:297,chr11:665405:665905:501:*:298
index_name,,,,,,,,,,,,,,,,,,,,,
GAAAGCCAGGCGAATA-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.238534,0.0,0.000000,0.0,0.0,0.0
TGTAAAGCAGGAATCG-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
CGTACGGGTAAAGCGG-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
AGTGTGGCAGTCTATG-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
TTTGACTTCGATTTGA-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.457913,0.000000,0.0,0.457913,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GGAACCACATGGAGGC-1_sSL0174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
TACAGCTAGCTAATTG-1_sSL0174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
TTGTCCCAGGAGTAAT-1_sSL0174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


In [12]:
ph_pilot_df = ph_pilot_df.join(ph_pilot.obs['donor_id'], how='inner')

del ph_pilot

ph_pilot_df

,chr11:632981:633481:501:*:276,chr11:635503:636003:501:*:277,chr11:636277:636777:501:*:278,chr11:636987:637487:501:*:279,chr11:637699:638199:501:*:280,chr11:638698:639198:501:*:281,chr11:639530:640030:501:*:282,chr11:640244:640744:501:*:283,chr11:641028:641528:501:*:284,chr11:641831:642331:501:*:285,...,chr11:650440:650940:501:*:290,chr11:655195:655695:501:*:291,chr11:657366:657866:501:*:292,chr11:658442:658942:501:*:293,chr11:659910:660410:501:*:294,chr11:661013:661513:501:*:295,chr11:662642:663142:501:*:296,chr11:663338:663838:501:*:297,chr11:665405:665905:501:*:298,donor_id
index_name,,,,,,,,,,,,,,,,,,,,,
GAAAGCCAGGCGAATA-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.238534,0.0,0.000000,0.0,0.0,0.0,SAMEA2547622
TGTAAAGCAGGAATCG-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,SAMEA3854308
CGTACGGGTAAAGCGG-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,SAMEA3854308
AGTGTGGCAGTCTATG-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,SAMEA2547622
TTTGACTTCGATTTGA-1_sSL0107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.457913,0.000000,0.0,0.457913,0.0,0.0,0.0,SAMEA2547622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GGAACCACATGGAGGC-1_sSL0174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,SAMEA2474458
TACAGCTAGCTAATTG-1_sSL0174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,SAMEA2518334
TTGTCCCAGGAGTAAT-1_sSL0174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,SAMEA2518334


In [13]:
ph_pilot_df = ph_pilot_df.groupby('donor_id').mean().T
ph_pilot_df = ph_pilot_df.rename_axis('peak_id')
ph_pilot_df

donor_id,SAMEA2420640,SAMEA2433566,SAMEA2445779,SAMEA2445784,SAMEA2459959,SAMEA2464819,SAMEA2474458,SAMEA2518322,SAMEA2518325,SAMEA2518334,...,SAMEA3973848,SAMEA3973854,SAMEA3974015,SAMEA3974241,SAMEA3974247,SAMEA3977051,SAMEA4089453,SAMEA4343052,SAMEA4451098,SAMEA4451109
peak_id,,,,,,,,,,,,,,,,,,,,,
chr11:632981:633481:501:*:276,0.000000,0.001109,0.000000,0.002860,0.021286,0.000000,0.000000,0.004175,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000432,0.000000,0.000000,0.000777,0.0,0.0,0.005316
chr11:635503:636003:501:*:277,0.002101,0.009687,0.000000,0.000000,0.052761,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.005003,0.0,0.0,0.003491
chr11:636277:636777:501:*:278,0.000000,0.004005,0.000000,0.007808,0.004850,0.000000,0.000000,0.003551,0.000000,0.000921,...,0.000000,0.0,0.006123,0.003746,0.000000,0.000000,0.002461,0.0,0.0,0.001802
chr11:636987:637487:501:*:279,0.010754,0.012582,0.000000,0.012427,0.012535,0.026906,0.049662,0.002169,0.000000,0.010590,...,0.044751,0.0,0.002532,0.011437,0.000000,0.000000,0.016099,0.0,0.0,0.011724
chr11:637699:638199:501:*:280,0.012822,0.024509,0.000000,0.010024,0.001826,0.018451,0.000000,0.015562,0.014320,0.003434,...,0.000000,0.0,0.003500,0.018897,0.010783,0.000000,0.013100,0.0,0.0,0.000000
chr11:638698:639198:501:*:281,0.014376,0.018946,0.000000,0.002030,0.038096,0.047601,0.000000,0.022787,0.000000,0.033155,...,0.060439,0.0,0.001377,0.006411,0.007219,0.031531,0.009293,0.0,0.0,0.009627
chr11:639530:640030:501:*:282,0.000718,0.008745,0.023389,0.031796,0.031428,0.033808,0.047307,0.005199,0.000000,0.014538,...,0.000000,0.0,0.002134,0.006987,0.000000,0.023385,0.013165,0.0,0.0,0.008313
chr11:640244:640744:501:*:283,0.001416,0.050120,0.000000,0.006039,0.004684,0.000000,0.000000,0.000000,0.000000,0.005379,...,0.000000,0.0,0.010239,0.003320,0.000000,0.000000,0.000995,0.0,0.0,0.006449
chr11:641028:641528:501:*:284,0.001343,0.007485,0.000000,0.002172,0.000000,0.000000,0.004935,0.005982,0.000000,0.004241,...,0.000000,0.0,0.000000,0.001919,0.013485,0.000000,0.003078,0.0,0.0,0.006836


In [14]:
samples_set = list(set(ph_pilot_df.columns) - excl)

ph_pilot_df = ph_pilot_df.loc[samples_set, :]
ph_pilot_df

KeyError: "None of [Index(['SAMEA2678742', 'SAMEA2433566', 'SAMEA3853161', 'SAMEA3974015',\n       'SAMEA2547633', 'SAMEA3973854', 'SAMEA2645805', 'SAMEA3973848',\n       'SAMEA4343052', 'SAMEA3977051', 'SAMEA3485962', 'SAMEA2593858',\n       'SAMEA3735541', 'SAMEA2547619', 'SAMEA2627577', 'SAMEA2625592',\n       'SAMEA2518325', 'SAMEA2711371', 'SAMEA4451109', 'SAMEA2547899',\n       'SAMEA3966392', 'SAMEA3974241', 'SAMEA2459959', 'SAMEA2547644',\n       'SAMEA2609971', 'SAMEA3962619', 'SAMEA3485958', 'SAMEA3847031',\n       'SAMEA2613912', 'SAMEA2464819', 'SAMEA2698309', 'SAMEA2570435',\n       'SAMEA2536410', 'SAMEA2536416', 'SAMEA2536413', 'SAMEA2547637',\n       'SAMEA2658084', 'SAMEA3974247', 'SAMEA3964906', 'SAMEA2707499',\n       'SAMEA3851971', 'SAMEA3854313', 'SAMEA3854308', 'SAMEA3968753',\n       'SAMEA3963885', 'SAMEA3965165', 'SAMEA2595426', 'SAMEA2698315',\n       'SAMEA2518334', 'SAMEA4089453', 'SAMEA3853122', 'SAMEA2445779',\n       'SAMEA3962389', 'SAMEA3962624', 'SAME

In [15]:
ph_pilot_df.round(6).to_csv("../data/datasets/hca_brain-organoids_processed/chromatin_accessibility/peak-matrix_rna-qc-cells_norm-reads-in-tss.tsv", sep='\t')

### CA PCs

In [16]:
grouped_ad = ad.AnnData(ph_pilot_df.T)
grouped_ad.obs_names = ph_pilot_df.columns.tolist()
grouped_ad.var_names = ph_pilot_df.index.tolist()

del ph_pilot_df

grouped_ad

AnnData object with n_obs × n_vars = 71 × 23

In [17]:
sc.pp.pca(grouped_ad)

In [18]:
ca_pcs = pd.DataFrame(grouped_ad.obsm['X_pca'][:, 0:20]).set_index(grouped_ad.obs_names)
ca_pcs.columns = [f'caPC_{str(i)}' for i in range(1, 21)]
ca_pcs = ca_pcs.T
ca_pcs = ca_pcs.rename_axis('id')
ca_pcs[samples_set].to_csv("../data/datasets/hca_brain-organoids_processed/covariates/ca_pcs.tsv", sep='\t')
ca_pcs

,SAMEA2420640,SAMEA2433566,SAMEA2445779,SAMEA2445784,SAMEA2459959,SAMEA2464819,SAMEA2474458,SAMEA2518322,SAMEA2518325,SAMEA2518334,...,SAMEA3973848,SAMEA3973854,SAMEA3974015,SAMEA3974241,SAMEA3974247,SAMEA3977051,SAMEA4089453,SAMEA4343052,SAMEA4451098,SAMEA4451109
id,,,,,,,,,,,,,,,,,,,,,
caPC_1,-0.002848,-0.002779,0.005161,0.008373,0.004187,-0.003168,0.000225,-0.002485,-0.002943,-0.003737,...,-0.006174,-0.009364,-0.007135,-0.005635,-0.004538,0.052058,-0.001288,-0.009364,-0.009364,-0.007622
caPC_2,-0.006798,-0.001515,-0.018295,-0.006514,-0.033389,-0.012387,-0.023169,0.000288,-0.003427,-0.004643,...,-0.007042,0.009206,0.004016,-0.000557,0.008023,0.003830,-0.001524,0.009206,0.009206,0.003083
caPC_3,0.004446,0.001390,0.025783,0.003383,0.029299,0.003675,0.023570,-0.000249,-0.002041,0.005140,...,0.000226,-0.020035,-0.014217,-0.008135,0.003951,-0.001454,-0.001917,-0.020035,-0.020035,-0.011626
caPC_4,-0.005016,0.002992,-0.005537,0.005690,0.029845,-0.002155,0.031737,-0.000307,-0.006350,-0.002587,...,-0.009970,-0.012984,-0.008564,0.000260,-0.005245,-0.008209,-0.001245,-0.012984,-0.012984,0.017662
caPC_5,-0.004901,0.015100,-0.028255,0.005098,0.015141,0.056350,0.008487,0.001455,-0.015741,0.011530,...,0.055055,-0.009406,-0.007500,0.002506,-0.003138,0.018077,0.004172,-0.009406,-0.009406,0.002062
caPC_6,-0.011264,-0.016881,0.015036,0.012600,0.027710,-0.010848,0.019756,-0.004176,-0.007624,0.000960,...,-0.029661,0.004165,-0.001385,-0.006383,-0.014051,-0.000250,0.000512,0.004165,0.004165,0.005167
caPC_7,0.002862,-0.009685,0.002738,-0.013091,0.046206,-0.004706,-0.021752,0.009241,0.003837,0.011502,...,0.024462,0.002093,-0.000163,-0.007171,0.013122,0.017984,-0.006033,0.002093,0.002093,0.009160
caPC_8,0.004301,0.032651,-0.003830,0.003288,0.029647,0.007922,-0.027950,0.004635,-0.001109,-0.004550,...,-0.025215,-0.003183,0.001027,0.004238,-0.005602,-0.010836,0.000971,-0.003183,-0.003183,-0.003576
caPC_9,-0.003951,0.000530,-0.007543,-0.003260,0.024778,-0.019776,0.021496,-0.007937,-0.012828,-0.000125,...,0.001015,0.001621,-0.002486,-0.005251,-0.021608,-0.021223,0.003761,0.001621,0.001621,0.009021


In [19]:
del grouped_ad
del ca_pcs

In [20]:
borgs_tile_mat.file.close()

## Filter features

In [21]:
# Subset to cell type
borgs_men = borgs_tile_mat[borgs_tile_mat.obs['celltype'] == 'Midbrain EN', :]
borgs_men

View of AnnData object with n_obs × n_vars = 24950 × 736845
    obs: 'BlacklistRatio', 'nDiFrags', 'nFrags', 'nMonoFrags', 'nMultiFrags', 'NucleosomeRatio', 'PassQC', 'PromoterRatio', 'ReadsInBlacklist', 'ReadsInPromoter', 'ReadsInTSS', 'Sample', 'TSSEnrichment', 'celltype', 'cellType', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'donor_id', 'clone', 'organoid', 'donor', 'leiden', 'stressed_vs_unstressed_celltypes', 'Batch', 'ReadsInPeaks', 'FRIP', 'barcode'
    var: 'chr', 'start', 'end', 'strand', 'peak_name', 'score', 'GC', 'nearest_gene', 'peak_type'

In [22]:
# Make df and aggregate across cells of the same donor

borgs_men_df = borgs_men.to_df().copy()



borgs_men_df = borgs_men_df.rename(columns=borgs_men.var['peak_name'].to_dict())
borgs_men_df = borgs_men_df.join(borgs_men.obs['donor_id'], how='inner')

borgs_men_df = borgs_men_df.groupby('donor_id').mean()
borgs_men_df = borgs_men_df.rename_axis('donor')

borgs_men_df = borgs_men[samples_set]

del borgs_men

borgs_men_df

KeyError: "Values ['SAMEA2678742', 'SAMEA2433566', 'SAMEA3853161', 'SAMEA3974015', 'SAMEA2547633', 'SAMEA3973854', 'SAMEA2645805', 'SAMEA3973848', 'SAMEA4343052', 'SAMEA3977051', 'SAMEA3485962', 'SAMEA2593858', 'SAMEA3735541', 'SAMEA2547619', 'SAMEA2627577', 'SAMEA2625592', 'SAMEA2518325', 'SAMEA2711371', 'SAMEA4451109', 'SAMEA2547899', 'SAMEA3966392', 'SAMEA3974241', 'SAMEA2459959', 'SAMEA2547644', 'SAMEA2609971', 'SAMEA3962619', 'SAMEA3485958', 'SAMEA3847031', 'SAMEA2613912', 'SAMEA2464819', 'SAMEA2698309', 'SAMEA2570435', 'SAMEA2536410', 'SAMEA2536416', 'SAMEA2536413', 'SAMEA2547637', 'SAMEA2658084', 'SAMEA3974247', 'SAMEA3964906', 'SAMEA2707499', 'SAMEA3851971', 'SAMEA3854313', 'SAMEA3854308', 'SAMEA3968753', 'SAMEA3963885', 'SAMEA3965165', 'SAMEA2595426', 'SAMEA2698315', 'SAMEA2518334', 'SAMEA4089453', 'SAMEA3853122', 'SAMEA2445779', 'SAMEA3962389', 'SAMEA3962624', 'SAMEA3967410', 'SAMEA2518322', 'SAMEA4451098', 'SAMEA2609965', 'SAMEA2420640', 'SAMEA2645814', 'SAMEA3962524', 'SAME

In [23]:
# Number of non-sparse peaks

thresh = 0.05
tot = len(borgs_men_df.columns)
npeaks_005 = sum((borgs_men_df != 0).mean() > thresh)
# npeaks_005 = borgs_men_df.loc[:, ((borgs_men_df != 0).mean() > thresh).values]
print(f'Total nr peaks: {tot:.2f}' \
      f'Filtered nr peaks: {npeaks_005}' \
      f'Proportion: {npeaks_005 / tot:.2f}'
    )

thresh = 0.10
npeaks_010 = sum((borgs_men_df != 0).mean() > thresh)
# npeaks_010 = borgs_men_df.loc[:, ((borgs_men_df != 0).mean() > thresh).values]
print(f'Total nr peaks: {tot:.2f}' \
      f'Filtered nr peaks: {npeaks_010}' \
      f'Proportion: {npeaks_010 / tot:.2f}'
    )

Total nr peaks: 736845.00Filtered nr peaks: 735534Proportion: 1.00


Total nr peaks: 736845.00Filtered nr peaks: 718101Proportion: 0.97


In [24]:
del borgs_men_df

In [25]:
# Density of CA

non0 = borgs_men.X.count_nonzero()
all_ = np.product(borgs_men.X.shape)
print(f'Density: {non0 / all_}')

Density: 0.00677796842522997


# Genotype

In [26]:
gt = pd.read_csv('../data/datasets/hca_brain-organoids_processed/covariates/genotype.tsv', sep='\t', header=0, index_col=0).rename_axis('snp_id')
gt = gt.rename(index=lambda x: 'chr' + str(x))
gt

,SAMEA2536416,SAMEA3854313,SAMEA3853161,SAMEA3966392,SAMEA2625592,SAMEA2698309,SAMEA3962624,SAMEA3965165,SAMEA2595426,SAMEA3962389,...,SAMEA2555017,SAMEA2420640,SAMEA4451109,SAMEA3448738,SAMEA2627141,SAMEA3485958,SAMEA2536413,SAMEA2464819,SAMEA2627567,SAMEA2645814
snp_id,,,,,,,,,,,,,,,,,,,,,
chr1_89599_A_T,1.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,1.0,1.0,...,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,0.0
chr1_94986_C_T,1.0,1.0,NaN,2.0,2.0,2.0,2.0,2.0,1.0,2.0,...,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0
chr1_94996_A_AT,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,1.0,2.0,...,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,0.0
chr1_95068_G_A,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,...,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0
chr1_96305_C_T,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr22_50780959_T_C,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,...,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0
chr22_50781276_G_A,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,...,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0
chr22_50783303_T_C,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,...,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0


In [27]:
dids_genotypes = set(gt.columns.tolist())

In [28]:
print(f'len dids_genotypes: {len(dids_genotypes)}')
print(f'len dids_borgs: {len(dids_borgs)}')
print(dids_genotypes - dids_borgs)
print(dids_borgs - dids_genotypes)

len dids_genotypes: 85
len dids_borgs: 72
{'SAMEA3963884', 'SAMEA4089454', 'SAMEA3754201', 'SAMEA2536404', 'SAMEA2627578', 'SAMEA2675459', 'SAMEA3851966', 'SAMEA2445790', 'SAMEA2627567', 'SAMEA2678738', 'SAMEA2609972', 'SAMEA2474454', 'SAMEA2518324', 'SAMEA2613897', 'SAMEA2555017'}
{'SAMEA2474458', 'SAMEA2555012'}


In [29]:
snps_set = gt.index

In [30]:
gt = gt[samples_set]
gt.to_csv('../data/datasets/hca_brain-organoids_processed/covariates/genotype_NA.tsv', sep='\t', na_rep='NaN')

# Covariates

## Population structure

In [31]:
gt_pcs = pd.read_csv("/omics/groups/OE0540/internal/projects/HCA_organoid_2/cemm_sabrina-20Jul2022/Genotypes/GT_PCs.tsv", sep="\t", index_col=0)
gt_pcs_matrixQTL = gt_pcs.T
gt_pcs_matrixQTL = gt_pcs_matrixQTL.rename_axis('id')

In [32]:
gt_pcs_matrixQTL

iid,SAMEA2536416,SAMEA3854313,SAMEA3853161,SAMEA3966392,SAMEA2625592,SAMEA2698309,SAMEA3962624,SAMEA3965165,SAMEA2595426,SAMEA3962389,...,SAMEA2555017,SAMEA2420640,SAMEA4451109,SAMEA3448738,SAMEA2627141,SAMEA3485958,SAMEA2536413,SAMEA2464819,SAMEA2627567,SAMEA2645814
id,,,,,,,,,,,,,,,,,,,,,
PC1,-141.934390,14.175615,-26.086426,-42.349370,-36.099750,-587.087340,-39.436108,-14.546681,-22.360580,4.674102,...,8.343557,-61.166600,8.252880,-14.844159,24.835945,-10.119088,37.227420,11.102370,-539.840700,60.762863
PC2,-963.927860,29.967335,12.337655,78.549210,69.155190,367.275100,11.223527,13.121081,16.850555,6.302846,...,30.577353,8.264520,11.945375,27.149958,39.365500,-14.613321,29.959509,-18.024128,-454.982480,-21.341936
PC3,-71.293526,-14.492196,8.423677,-1.863944,54.365560,587.046800,7.243265,7.121950,43.723200,-0.185648,...,-31.239319,27.960432,49.424560,19.483690,17.270079,35.091404,7.322402,9.814706,-640.335300,40.542133
PC4,18.946665,-38.462044,-15.730290,17.993840,8.240096,372.022800,19.611492,-17.582450,-1.413503,35.015190,...,12.754401,66.276980,46.583675,1.502355,9.076994,-3.814449,4.514056,14.720867,496.289370,-49.056087
PC5,-371.363400,18.293554,12.673084,-35.727848,-5.430482,49.229893,16.133036,-32.255928,30.050665,3.094576,...,90.688050,0.795895,-17.147549,-26.146770,-8.983247,2.491865,20.821611,16.440897,742.540800,9.650810
PC6,-577.124300,-29.827420,-49.799580,-13.191879,7.309116,-182.055220,-32.406673,-40.839900,-39.224907,22.390940,...,22.033606,32.240032,33.691920,-25.984467,8.160939,4.828308,-4.850675,37.459100,-345.033750,-55.715916
PC7,1123.644800,-21.026545,-11.844399,-20.626287,-14.493244,-386.220730,23.316786,-15.643868,10.394878,-26.352718,...,31.516148,24.833984,-7.336715,-15.710847,-16.682304,-21.255926,-5.095608,-9.256178,-119.823500,-34.933605
PC8,-458.672880,16.555382,18.833464,25.677296,-20.599740,-129.463040,-19.541563,-1.930764,-32.527310,13.858808,...,-3.647696,36.857200,-11.409421,27.067438,-12.322556,-12.550730,-4.225716,16.599375,938.962770,13.931620
PC9,-610.459530,-36.159980,23.225570,-9.546488,6.205075,-532.290700,-9.519834,17.155119,-11.905414,-11.894598,...,-24.272205,-70.586040,-43.480988,-25.920094,-31.535606,-14.428098,-12.799575,-28.210615,529.441300,47.833714


In [33]:
parent_dir = "/home/fichtner/projects/footprintQTL/data/datasets/hca_brain-organoids_processed/covariates/"

if not os.path.isdir(parent_dir):
    os.makedirs(parent_dir)

In [34]:
gt_pcs_matrixQTL[samples_set].to_csv("/home/fichtner/projects/footprintQTL/data/datasets/hca_brain-organoids_processed/covariates/genotype_pcs.tsv", sep='\t')

In [35]:
dids_cov = set(gt_pcs_matrixQTL.columns.tolist())

In [36]:
print(f'len dids_cov: {len(dids_cov)}')
print(f'len dids_borgs: {len(dids_borgs)}')
print(dids_cov - dids_borgs)
print(dids_borgs - dids_cov)

len dids_cov: 85
len dids_borgs: 72
{'SAMEA3963884', 'SAMEA4089454', 'SAMEA3754201', 'SAMEA2536404', 'SAMEA2627578', 'SAMEA2675459', 'SAMEA3851966', 'SAMEA2445790', 'SAMEA2627567', 'SAMEA2678738', 'SAMEA2609972', 'SAMEA2474454', 'SAMEA2518324', 'SAMEA2613897', 'SAMEA2555017'}
{'SAMEA2474458', 'SAMEA2555012'}


In [37]:
del gt_pcs
del gt_pcs_matrixQTL

## SNP locations

In [38]:
snps = pd.read_csv("../data/datasets/hca_brain-organoids_processed/covariates/genotype.vcf",
                   sep="\t",
                   comment="#",
                   header=None,
                   names=["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT", "SAMEA2536416", "SAMEA3854313", "SAMEA3853161", "SAMEA3966392", "SAMEA2625592", "SAMEA2698309", "SAMEA3962624", "SAMEA3965165", "SAMEA2595426", "SAMEA3962389", "SAMEA3973854", "SAMEA3847031", "SAMEA2547622", "SAMEA3854308", "SAMEA3964906", "SAMEA2536410", "SAMEA3735541", "SAMEA2590621", "SAMEA2698315", "SAMEA2445784", "SAMEA3973848", "SAMEA3962524", "SAMEA3485962", "SAMEA2609965", "SAMEA3851971", "SAMEA4089454", "SAMEA2613912", "SAMEA3963885", "SAMEA3963884", "SAMEA2613897", "SAMEA4089453", "SAMEA3851966", "SAMEA3974241", "SAMEA4343052", "SAMEA3962619", "SAMEA3853122", "SAMEA3974015", "SAMEA2518325", "SAMEA2433566", "SAMEA2459959", "SAMEA2675459", "SAMEA2682671", "SAMEA2645805", "SAMEA2711371", "SAMEA2593858", "SAMEA2445790", "SAMEA2518322", "SAMEA2547619", "SAMEA2627577", "SAMEA2547637", "SAMEA3977051", "SAMEA2445779", "SAMEA2609971", "SAMEA3974247", "SAMEA2658107", "SAMEA3754205", "SAMEA2547644", "SAMEA2678742", "SAMEA2474454", "SAMEA2627566", "SAMEA2518334", "SAMEA3967410", "SAMEA2536404", "SAMEA2627578", "SAMEA2518324", "SAMEA3968753", "SAMEA2547899", "SAMEA3754201", "SAMEA2570435", "SAMEA2658084", "SAMEA4451098", "SAMEA2678738", "SAMEA2547633", "SAMEA2609972", "SAMEA2707499", "SAMEA2555017", "SAMEA2420640", "SAMEA4451109", "SAMEA3448738", "SAMEA2627141", "SAMEA3485958", "SAMEA2536413", "SAMEA2464819", "SAMEA2627567", "SAMEA2645814"],
                   index_col=False
                  )

snps = snps[['ID', 'CHROM', 'POS']].rename(columns={'ID': 'snp_id', 'CHROM': 'chr', 'POS': 'pos'})
snps = snps.set_index('snp_id')

In [39]:
snps

,chr,pos
snp_id,,
chr1_30903_ATCTC_A,chr1,30903
chr1_65797_T_C,chr1,65797
chr1_66596_AT_A,chr1,66596
chr1_67353_A_G,chr1,67353
chr1_69511_A_G,chr1,69511
...,...,...
chr22_50783303_T_C,chr22,50783303
chr22_50783438_G_C,chr22,50783438
chr22_50783506_G_T,chr22,50783506


In [40]:
snps = snps.loc[snps_set]
snps.to_csv("../data/datasets/hca_brain-organoids_processed/covariates/snp_locations.tsv", sep='\t')

In [41]:
del snps